In [1]:
%pwd

'C:\\Users\\DELL\\OneDrive\\Desktop\\MLOps\\chicken_disease_classification\\research'

In [2]:
import os
os.chdir('../')
%pwd

'C:\\Users\\DELL\\OneDrive\\Desktop\\MLOps\\chicken_disease_classification'

In [18]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: str

In [19]:
from CDC.constants import *
from CDC.utils.common import read_yaml, create_directories

In [24]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
        
    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_checkpoint_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_checkpoint_dir),
            Path(config.tensorboard_root_log_dir)
        ])
        
        prepare_callbacks_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=config.checkpoint_model_filepath
        )
        
        return prepare_callbacks_config

In [25]:
import os
import urllib.request as request
from zipfile import ZipFile
import time
import tensorflow as tf

In [26]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config
        
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime('%Y-%m-%d-%h-%M-%S')
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f'tb_logs_at_{timestamp}'
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )
    
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_ckpt_callbacks,
            self._create_tb_callbacks            
        ]

In [28]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
except Exception as e:
    raise e

[2023-11-27 18:35:39,011: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-27 18:35:39,017: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-27 18:35:39,019: INFO: common: created directory at: artifacts]
[2023-11-27 18:35:39,023: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2023-11-27 18:35:39,025: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
